In [ ]:
from pymysql import *
from datetime import *
import tkinter as tk
from tkinter import ttk

class Utility:
    def connection(self):
        con = connect(db='bank', user='root', password='root', host='localhost', port=3306)
        return con

    def todaydate(self):
        dt = datetime.now()
        ndate = str(dt.year) + '-' + str(dt.month) + '-' + str(dt.day)
        return ndate

    def openaccount(self):
        con = self.connection()
        cur = con.cursor()
        acno = int(input('Account no'))
        cur.execute('select * from master where acno=%d' % (acno))
        data = cur.fetchall()
        if len(data) == 1:
            print('Duplicate Account Number')
            con.close()
        else:
            cname = input('Customer Name')
            amt = float(input('Amount for opening account'))
            ndate = self.todaydate()
            i = cur.execute("insert into master values(%d,'%s','%s','%s',%f)" % (acno, cname, ndate, ndate, amt))
            if i == 1:
                j = cur.execute("insert into trans values(%d,'%s','%s',%f)" % (acno, ndate, 'D', amt))
                if j == 1:
                    con.commit()
                    print('Account is now opened')
                    con.close()

    def depositamount(self):
        con = self.connection()
        cur = con.cursor()
        acno = int(input('Account no'))
        cur.execute('select * from master where acno=%d' % (acno))
        data = cur.fetchall()
        if len(data) == 1:
            amt = float(input('Deposit amount'))
            ndate = self.todaydate()
            i = cur.execute("update master set balance=balance+%f,last_transaction_date='%s' where acno=%d" % (amt, ndate, acno))
            if i == 1:
                j = cur.execute("insert into trans values(%d,'%s','%s',%f)" % (acno, ndate, 'D', amt))
                if j == 1:
                    con.commit()
                    con.close()
                    print('Amount deposited')
        else:
            print('Account number not in bank')
            con.close()

    def withdrawamount(self):
        con = self.connection()
        cur = con.cursor()
        acno = int(input('Account NO '))
        cur.execute("select * from master where acno=%d" % (acno))
        data = cur.fetchall()
        if len(data) == 1:
            amt = float(input('Withdraw Amount '))
            ndate = self.todaydate()
            i = cur.execute("update master set balance=balance-%f,last_transaction_date='%s' where acno=%d" % (amt, ndate, acno))
            if i == 1:
                j = cur.execute("insert into trans values(%d,'%s','%s',%f)" % (acno, ndate, 'W', amt))
                if j == 1:
                    con.commit()
                    con.close()
                    print('Amount Withdrawn')
        else:
            print('Account Number Not in Bank')
            con.close()

    def showbalance(self):
        con = self.connection()
        cur = con.cursor()
        acno = int(input('Account no'))
        if acno == 0:
            print("Account does not exist")
        else:
            cur.execute("select balance from master where acno=%d" % (acno))
            mdata = cur.fetchall()
            cur.execute("select * from trans where acno=%d" % (acno))
            tdata = cur.fetchall()
            con.close()
            for xdata in tdata:
                for xd in xdata:
                    print(xd, end='\t')
                print()
            print("Balance in account no", acno, 'is', mdata[0][0])

    def closeAccount(self):
        con = self.connection()
        cur = con.cursor()
        acno = int(input('Account NO '))
        j = cur.execute("delete from trans where acno=%d" % (acno))
        if j >= 1:
            i = cur.execute("delete from master where acno=%d" % (acno))
            if i == 1:
                con.commit()
                print('Account Closed')
                con.close()

    def showAccountData(self):
        con = self.connection()
        cur = con.cursor()
        acno = int(input('Enter Account No: '))
        print("\nYour Account Details:")
        cur.execute("SELECT * FROM master WHERE acno = %d" % (acno))
        account_data = cur.fetchall()
        if account_data:
            for row in account_data:
                print(row)
        else:
            print("No account found.")

        print("\nYour Transaction History:")
        cur.execute("SELECT * FROM trans WHERE acno = %d" % (acno))
        trans_data = cur.fetchall()
        if trans_data:
            for row in trans_data:
                print(row)
        else:
            print("No transactions found.")

        cur.execute("SELECT * FROM account_closed WHERE acno = %d" % (acno))
        closed_data = cur.fetchone()
        if closed_data:
            print("\nStatus: Your account has been DEACTIVATED.")
        else:
            print("\nStatus: Your account is ACTIVE.")
        con.commit()
        con.close()

    def Main(self):
        ans = 'y'
        while ans.lower() == 'y':
            print('1. Open Account')
            print('2. Deposit Amount')
            print('3. Withdraw Amount')
            print('4. Show Balance')
            print('5. Close Account')
            print('6. Show Data by Account No')
            print('7. View Tables in GUI')
            print('8. Exit')
            print('Enter Choice (1-8):')
            ch = int(input())
            if ch == 1:
                self.openaccount()
            elif ch == 2:
                self.depositamount()
            elif ch == 3:
                self.withdrawamount()
            elif ch == 4:
                self.showbalance()
            elif ch == 5:
                self.closeAccount()
            elif ch == 6:
                self.showAccountData()
            elif ch == 7:
                self.launch_gui_viewer()
            elif ch == 8:
                break
            print('Do you want to continue (y/n)?')
            ans = input()

    def launch_gui_viewer(self):
        viewer_window = tk.Tk()
        TableViewer(viewer_window)
        viewer_window.mainloop()

class TableViewer:
    def __init__(self, root):
        self.root = root
        self.root.title("Bank Table Viewer")
        self.root.geometry("800x500")

        self.table_var = tk.StringVar()
        self.table_dropdown = ttk.Combobox(root, textvariable=self.table_var)
        self.table_dropdown['values'] = ['master', 'trans', 'account_closed']
        self.table_dropdown.grid(row=0, column=0, padx=10, pady=10)
        self.table_dropdown.current(0)

        self.load_btn = tk.Button(root, text="Load Data", command=self.load_data)
        self.load_btn.grid(row=0, column=1, padx=10, pady=10)

        self.tree = ttk.Treeview(root)
        self.tree.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

    def load_data(self):
        table_name = self.table_var.get()

        # Use pymysql here instead of mysql.connector
        con = connect(db='bank', user='root', password='root', host='localhost', port=3306)
        cur = con.cursor()

        cur.execute(f"SELECT * FROM {table_name}")
        rows = cur.fetchall()
        columns = [desc[0] for desc in cur.description]

        self.tree.delete(*self.tree.get_children())
        self.tree["columns"] = columns
        self.tree["show"] = "headings"

        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)

        for row in rows:
            self.tree.insert("", "end", values=row)

        cur.close()
        con.close()

# Launch the app
if __name__ == "__main__":
    obj = Utility()
    obj.Main()


1. Open Account
2. Deposit Amount
3. Withdraw Amount
4. Show Balance
5. Close Account
6. Show Data by Account No
7. View Tables in GUI
8. Exit
Enter Choice (1-8):


 7


Do you want to continue (y/n)?


 y


1. Open Account
2. Deposit Amount
3. Withdraw Amount
4. Show Balance
5. Close Account
6. Show Data by Account No
7. View Tables in GUI
8. Exit
Enter Choice (1-8):


...


Your Account Details:
(111, 'Arpit', datetime.date(2025, 8, 4), datetime.date(2025, 8, 4), 46000.0)

Your Transaction History:
(111, datetime.date(2025, 8, 4), 'D', 46000.0)

Status: Your account is ACTIVE.
Do you want to continue (y/n)?


 y


1. Open Account
2. Deposit Amount
3. Withdraw Amount
4. Show Balance
5. Close Account
6. Show Data by Account No
7. View Tables in GUI
8. Exit
Enter Choice (1-8):


 7


Do you want to continue (y/n)?
